- Average Annual Household Transportation Costs: Out-of-pocket expenses including tolls, parking, and transit fares

    - These costs do not include other vehicle operating and purchase costs.

    - Costs are annualized using factor of 300, as used in the 2018 RTP.


In [1]:
import os, sys, shutil
# sys.path.append(os.path.join(os.getcwd(),"inputs"))
# sys.path.append(os.path.join(os.getcwd(),r"../../../scripts/summarize/standard"))
# sys.path.append(os.path.join(os.getcwd(),r'../../..'))
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
from shapely import wkt
# from input_configuration import base_year
# from emme_configuration import *
# from standard_summary_configuration import *
pd.options.mode.chained_assignment = None
import os, sys, time, shutil
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None
import h5py
import toml
config = toml.load(os.path.join(os.getcwd(),'../../../../configuration/input_configuration.toml'))
summary_config = toml.load(os.path.join(os.getcwd(),'../../../../configuration/summary_configuration.toml'))


%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
# relative_path = '../../../outputs/agg/dash'
output_path = os.path.join(summary_config['sc_run_path'], summary_config["output_folder"])
survey_path = os.path.join(summary_config['sc_run_path'], summary_config["survey_folder"])

In [3]:
df = pd.read_csv(os.path.join(output_path,'agg/dash/person_cost.csv'))
df_hh = pd.read_csv(os.path.join(output_path,'agg/dash/hh_geog.csv'))

annual_factor = 300

df['is_rgc'] = 0
df.loc[df['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_hh['is_rgc'] = 0
df_hh.loc[df_hh['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

In [4]:
df = df[(((df['mode'].isin(['SOV','HOV2','HOV3+']) & (df['dorp'] == 1)) 
          | (~df['mode'].isin(['SOV','HOV2','HOV3+']))))]

In [5]:
pd.options.display.float_format = '${:0,.0f}'.format

## County

In [6]:
_df = pd.DataFrame(df.groupby('hh_county').sum()['travcost_wt']/df_hh.groupby('hh_county').sum()['hhexpfac'])*annual_factor
_df.columns = ['Average Annual Costs']
_df.loc['Region'] = df['travcost_wt'].sum()/df_hh['hhexpfac'].sum()*annual_factor
_df

,Average Annual Costs
hh_county,
King,"$2,152"
Kitsap,"$2,129"
Pierce,"$2,088"
Snohomish,"$2,369"
Region,"$2,173"


## Regional Growth Center

In [7]:
_df = pd.DataFrame(df.groupby('is_rgc').sum()['travcost_wt']/df_hh.groupby('is_rgc').sum()['hhexpfac'])*annual_factor
_df.columns = ['Average Annual Costs']
_df

,Average Annual Costs
is_rgc,
0,"$2,345"
1,"$1,006"


In [8]:
_df = pd.DataFrame(df.groupby('hh_rgc').sum()['travcost_wt']/df_hh.groupby('hh_rgc').sum()['hhexpfac'])*annual_factor
_df.columns = ['Average Annual Costs']
_df

,Average Annual Costs
hh_rgc,
Auburn,"$1,880"
Bellevue,"$1,218"
Bothell Canyon Park,"$2,225"
Bremerton,"$1,131"
Burien,"$1,839"
Everett,"$1,399"
Federal Way,"$2,332"
Greater Downtown Kirkland,"$2,112"
Kent,"$1,672"


## Regional Geography

In [9]:
_df = pd.DataFrame(df.groupby('hh_rg_proposed').sum()['travcost_wt']/df_hh.groupby('hh_rg_proposed').sum()['hhexpfac'])*annual_factor
_df.columns = ['Average Annual Costs']
_df

,Average Annual Costs
hh_rg_proposed,
CitiesTowns,"$3,138"
Core,"$2,470"
HCT,"$2,462"
Metro,"$1,663"


## Equity Geography

In [10]:
results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if df_hh[df_hh[equity_geog+geog_type] == 1].sum()['hhexpfac'] > 0:
            _df = (df[df[equity_geog+geog_type] == 1].sum()['travcost_wt']/df_hh[df_hh[equity_geog+geog_type] == 1].sum()['hhexpfac'])*annual_factor
            results_df.loc[equity_geog+geog_type,'costs'] = _df
        
inv_results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if df_hh[df_hh[equity_geog+geog_type] == 1].sum()['hhexpfac'] > 0:
            _df = (df[df[equity_geog+geog_type] == 0].sum()['travcost_wt']/df_hh[df_hh[equity_geog+geog_type] == 0].sum()['hhexpfac'])*annual_factor
            inv_results_df.loc[equity_geog+geog_type,'costs'] = _df
        
# inv_results_df.index = ['NOT in '+i for i in results_df.index] 
_df = results_df.merge(inv_results_df, left_index=True, right_index=True)
_df.columns = ['Inside Equity Geog','Outside Equity Geog']
_df[~_df.isnull()]

,Inside Equity Geog,Outside Equity Geog
hh_youth_reg,"$2,632","$1,887"
hh_elderly_reg,"$2,195","$2,154"
hh_english_reg,"$2,332","$2,066"
hh_racial_reg,"$2,144","$2,202"
hh_racial_50,"$2,167","$2,176"
hh_poverty_reg,"$1,981","$2,307"
hh_poverty_50,"$1,511","$2,190"
hh_disability_reg,"$2,003","$2,305"
